In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


Добавляем основные импорты.

In [ ]:
import tqdm
import shutil
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

Наиболее частая архитектура свёрточной нейронной сети выглядит так:
INPUT -> [[CONV -> RELU]*N -> POOL?] * M -> [FC -> RELU]*K -> FC 


In [ ]:
img_width, img_height =150,150   #размер изображения 
test_size = 0.15
input_shape=(img_width,img_height, 3)# для x×x RGB-картинок
EPOCHS = 15 
BATCH_SIZE = 120



In [ ]:
CONTENT_DIR = '/kaggle/content'
TRAIN_DIR = CONTENT_DIR + '/train'
TRAIN_DIR_DOG = TRAIN_DIR + '/dog'
TRAIN_DIR_CAT = TRAIN_DIR + '/cat'

VALID_DIR = CONTENT_DIR + '/valid'
VALID_DIR_DOG = VALID_DIR + '/dog'
VALID_DIR_CAT = VALID_DIR + '/cat'

if not os.path.exists(CONTENT_DIR):
    import zipfile
    with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r') as zipf:
        zipf.extractall(CONTENT_DIR)

    
    img_filenames = os.listdir(TRAIN_DIR)
    print('Num of images:', len(img_filenames))

    dog_filenames = [fn for fn in img_filenames if fn.startswith('dog')]
    cat_filenames = [fn for fn in img_filenames if fn.startswith('cat')]

    dataset_filenames = train_test_split(
        dog_filenames, cat_filenames, test_size=test_size, shuffle=True, random_state=42
    )

    train_dog_total, valid_dog_total, train_cat_total, valid_cat_total = [len(fns) for fns in dataset_filenames]
    train_total = train_dog_total + train_cat_total
    valid_total = valid_dog_total + valid_cat_total
    print('Train: {}, test: {}'.format(train_total, valid_total))

    # Move images
    make_dirs = [TRAIN_DIR_DOG, VALID_DIR_DOG, TRAIN_DIR_CAT, VALID_DIR_CAT]
    for dir, fns in zip(make_dirs, dataset_filenames):
        os.makedirs(dir, exist_ok=True)
        for fn in tqdm.tqdm(fns):
            shutil.move(os.path.join(TRAIN_DIR, fn), dir)
        print('elements in {}: {}'.format(dir, len(os.listdir(dir))))

Можно увеличить объем данных засчет манипуляций с уже имеющимися данными:отражение, зум.  

In [ ]:
image_generator = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255
)
train_data = image_generator.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary'
)

Сделаем входной слой по аналогу с CIFAR-10 с функцией активации ReLu. Мной была выбрана функция активации ReLu, так как функция является хорошим аппроксиматором и любая функция может быть аппроксимирована комбинацией ReLu. 
Еще одним достоинством является разреженность активации. Использование сигмоиды или гиперболического тангенса будет влечь за собой активацию всех нейронов аналоговым способом. Это означает, что почти все активации должны быть обработаны для описания выхода сети. Другими словами, активация плотная, а это затратно. В идеале мы хотим, чтобы некоторые нейроны не были активированы, это сделало бы активации разреженными и эффективными.
После добавим слой подвыборки для уменьшения размерности входного представления. Наиболее популярные варианты конфигурации: 2х2 (F=2) и шагом 2 (S=2), но исключает 75% активаций входного представления (из-за сэмплирования, в два раза по ширине и высоте) и 3х3 (размер фильтр) и 2 (размер шага). Необходимо учитывать, что подвыборка представляет собой очень агрессивную функцию при применении которой теряется информация по признакам.  

После используем один слой для классификации и функцию активации.

In [ ]:
model = Sequential()
#входной сверточный слой, параметры подобраны на подобии  CIFAR-10
model.add(Conv2D(filters = 32, kernel_size = (3, 3), input_shape=input_shape, activation = 'relu', strides = (1,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', strides = (1,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', strides = (1,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1))
model.add(Activation('sigmoid'))          


Немного о выборе ядра для сверточного слоя.
Единственный 7х7 свёрточный слой будет содержать (Cх(7х7хС)) = 49xСxC параметров, в то время как три свёрточных слоя 3х3 будут содержать только 3х(Сх(3х3хС)) = 27хСхС параметра, к тому же на каждом из слоев можно использовать функцию активации. Интуитивно становится понятно, что последовательное размещение свёрточных слоёв с небольшими фильтрами вместо размещения одного свёрточного слоя с большим размером фильтра, позволяет выделить больше значимых признаков из входных данных с меньшим количеством параметров. 


Валидация

In [ ]:
valid_generator = ImageDataGenerator(rescale=1./255)
valid_data = valid_generator.flow_from_directory(
    directory=VALID_DIR,
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Об оптимизаторах: https://habr.com/ru/post/318970/

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(
    generator=train_data,
    steps_per_epoch=(train_total + BATCH_SIZE - 1) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=valid_data,
    validation_steps=(valid_total + BATCH_SIZE - 1) // BATCH_SIZE,
)


In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), history.history['accuracy'], label='train')
plt.plot(range(EPOCHS), history.history['val_accuracy'], label='valid')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), history.history['loss'], label='train')
plt.plot(range(EPOCHS), history.history['val_loss'], label='valid')
plt.legend(loc='upper right')
plt.title('Loss (sparse_categorical_crossentropy)')

plt.show()
model.save("/kaggle/output/kaggle/working")